In [1]:
### define variable threshold ### VTA = variable threshold analysis
def VTA(threshold,pegeldata):
    threshold=80
    p_threshold=(100-threshold)/100
    df_mt_rv=pd.DataFrame()
    plot=False
    
    for j in df_rv.columns:
        months = pd.date_range(start='1901-01-01', periods=len(df_rv), freq='D')
        data=df_rv[j].tolist()
        #print(len(data))
        df = pd.DataFrame({'Date': months, 'Value': data})
        df['Date']=pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        df['D'] = df.index.dayofyear
        df['day_of_year'] = df.index.dayofyear
        # Berechnung des 80. Perzentils für jeden Monat
        df_valid=df[df['Value']!=-999]
        #print(df_valid.dtypes)
        mt = df_valid.groupby('D')['Value'].quantile(p_threshold)
        list_b=df_valid.groupby('D')['Value'].min()
        list_a=df_valid.groupby('D')['Value'].max()
        mt=mt.tolist()
        #print(len(mt))
        threshold_df = pd.DataFrame({'day_of_year': np.arange(1, len(mt)+1), 'threshold': mt})
        df = df.merge(threshold_df, on='day_of_year', how='left')
        mt=mt*31
        mt=mt[0:366]
        #print(river_ids[mst_valid_ids.index(j)])
        df_mt_rv[j]=df['threshold']
    return df_mt_rv

In [ ]:
import datetime


def calculate_drought(q_datas,MT,CDR):
    df=pd.DataFrame()
    ID=[]
    events=[]#0,1,2,3
    duration=[]#unit=month
    mean_deficit=[]#mean_deficit kleiner als 0
    max_deficit=[]
    sum_deficit=[]
    start_time=[]
    end_time=[]
    max_time=[]
    deficit_list=[]
    #duration_list=[]
    deta=[]
    Qs=[]
    amean=q_datas.tolist()
    
    days = pd.date_range(start='1901-01-01', periods=len(amean), freq='D')
    df_data = pd.DataFrame({'Date': days, 'Value': amean})
    df_data['Date']=pd.to_datetime(df_data['Date'])
    df_data.set_index('Date', inplace=True)
    
    dflag=False #drought flag
    i=0
    index_a=0
    amean=[999999 if int(i) ==-999 else i for i in  amean]
    s_time=0
    d=0
    for data in amean:
        
        MNQ95=MT[index_a]

        
        if data<MNQ95:
            if dflag==False:#drought starts
                dflag=True 
                ID.append(i)
                i=i+1
                
                deta.append(index_a-(s_time+d))
                s_time=index_a
                start_time.append(s_time)
                
                #print(s_time)
                d=1
            
                Q=[]
                Q.append(data)
                deficit=[]
                deficit.append(MNQ95-data)
            elif dflag==True: #during the drought
                deficit.append(MNQ95-data)
                Q.append(data)
                d=d+1
        elif data>=MNQ95:
            if dflag==True: #drought ends
                duration.append(d)
                mean_deficit.append(sum(deficit)/len(deficit))
                sum_deficit.append(sum(deficit))
                max_deficit.append(max(deficit))
                index_m=index_a+deficit.index(max(deficit))
                max_time.append(index_m)
                deficit_list.append(deficit)
                end_time.append(s_time+d)
                Qs.append(min(Q))
                #maxd=-max(deficit)
    
                dflag=False
        if data==amean[-1]:
            if dflag==True:
                duration.append(d)
                mean_deficit.append(sum(deficit)/len(deficit))
                sum_deficit.append(sum(deficit))
                max_deficit.append(max(deficit))
                index_m=index_a+deficit.index(max(deficit))
                max_time.append(index_m)
                deficit_list.append(deficit)
                end_time.append(s_time+d)
                Qs.append(min(Q))
      
                dflag=False
        if CDR == True:
            if dflag==True:
                if df_data.index.is_month_end[index_a]:
                    duration.append(d)
                    mean_deficit.append(sum(deficit)/len(deficit))
                    sum_deficit.append(sum(deficit))
                    max_deficit.append(max(deficit))
                    index_m=index_a+deficit.index(max(deficit))
                    max_time.append(index_m)
                    deficit_list.append(deficit)
                    end_time.append(s_time+d)
                    Qs.append(min(Q))
          
                    dflag=False

        index_a=index_a+1
        
    from datetime import date
    from dateutil.relativedelta import relativedelta
    
    def m2y(m):
        startdatum = date(1901, 1, 1)  # Reference start date
        neudate = startdatum + relativedelta(days=m)  # Add months
        years = neudate.year - startdatum.year  # Calculate years
        months = neudate.month - startdatum.month  # Calculate remaining months
    
        if months < 0:  # Handle negative months (when crossing a year)
            years -= 1
            months += 12
        years=1901+years
        months=months+1
        return years, months,neudate  # Return as (years, months)
    

                
    
    time_months=[m2y(i)[1] for i in start_time]    
    time_years=[m2y(i)[0] for i in start_time]  
    time_date=[m2y(i)[2] for i in start_time]
    end_date=[m2y(i)[2] for i in end_time]

    time_date_max =[m2y(i)[2] for i in max_time]
    time_month_max =[m2y(i)[1] for i in max_time]

    df['ID']=ID
    df['duration']=duration
    df['time_deta']=deta#[i if i<=365 else 0 deta]
    df['mean_deficit']=mean_deficit
    df['max_deficit']=max_deficit
    df['sum_deficit']=sum_deficit
    df['list_deficit']=deficit_list
    df['Qmin']=Qs
    df['start_date']=time_date
    df['start_year']=time_years
    df['start_month']=time_months
    df['max_date']=time_date_max
    df['max_month']=time_month_max
    df['end_date']=end_date
    df['start_year']=time_years
    df['start_month']=time_months

    
    
                
    return df